# 2020 T2 COMP9417 Group Project

## Group Member  
* Shu Yang (z5172181)  
* Yue Qi (z5219951)  
* Tim Luo  
* Yixiao Zhan (z5210796)


## Topic Overview
In a world… where movies made an estimated $41.7 billion in 2018, the film industry is more popular than ever. But what movies make the most money at the box office? How much does a director matter? Or the budget? For some movies, it's "You had me at 'Hello.'" For others, the trailer falls short of expectations and you think "What we have here is a failure to communicate."

In this competition, you're presented with metadata on over 7,000 past films from The Movie Database to try and predict their overall worldwide box office revenue. Data points provided include cast, crew, plot keywords, budget, posters, release dates, languages, production companies, and countries. You can collect other publicly available data to use in your model predictions, but in the spirit of this competition, use only data that would have been available before a movie's release.

A link to this Kaggle competition is provided below 
> https://www.kaggle.com/c/tmdb-box-office-prediction/overview

## Date pre-processing 
Import the relevant packages we'll need for this project.

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
import json
import ast
import math
from collections import Counter

Read in training and test data  


In [ ]:
train = pd.read_csv("./Data/train.csv")
test = pd.read_csv("./Data/test.csv")

### Yue's part
Yue is working on data pre-processing for columns from id to production company

In [ ]:
# Add your code here

### Yixiao's part
Yixiao is working on data pre-processing for columns from production country to production company keywords

#### Production countries

In [ ]:
# Separate and extract country name from production_countries
train['production_countries_count'] = train['production_countries'].apply(lambda x: len(ast.literal_eval(x)) if x == x else 0)
train['all_produced_countries'] = train['production_countries'].apply(lambda x: ",".join(sorted([i['name'] for i in ast.literal_eval(x)])) if x == x else '')

# Calculate mean revenue for these movies
revenue_means = {}
for value in train['all_produced_countries'].unique():
    query = "all_produced_countries=='" + value + "'"
    revenue_means[value] = round(train.query(query)['revenue'].mean(),2)

# Print the top 10 mean revenue
counter = Counter(revenue_means)
print ("TOP 10 revenue by production countries\n")
for v in counter.most_common(10):
    countries, mean_revenue = v
    print("Movie produced from " + countries + " has mean revenue " + str(mean_revenue))

We see from the above value counts that most movies are made by Czech Republic,United Arab Emirates,United States of America

In [ ]:
# Add columns produced_by_USA if needed

# train["produced_by_USA"] = train["all_countries"].apply(lambda x: 1 if "United States of America" in x else 0)

#### Release date
Break release into month, day and year and derive release_dayofweek from the date
Add all these information to separate columns in train dataframe

In [ ]:
# Release date data preprocessing
# Split release month, day, year into sepearte columns
train[["release_month","release_day","release_year"]] = train["release_date"].str.split("/",expand=True).replace(np.nan, -1).astype(int)
# Change release year to 4 digits year
train.loc[ (train['release_year'] <= 19) & (train['release_year'] < 100), "release_year"] += 2000
train.loc[ (train['release_year'] > 19)  & (train['release_year'] < 100), "release_year"] += 1900
release_date = pd.to_datetime(train["release_date"])
# Add the derived release weekday name to train
train["release_dayofweek"] = release_date.dt.weekday_name

In [ ]:
# Encode release_dayofweek
encoder = preprocessing.LabelBinarizer()
release_dayofweek_transformed = encoder.fit_transform(train["release_dayofweek"])
release_dayofweek = pd.DataFrame(release_dayofweek_transformed)
# Add to train dataframe
train = pd.concat([train,release_dayofweek], axis=1)

# Rename column name for easier use
train.rename(columns={0 : "released_on_" + str(encoder.classes_[0]), 1 : "released_on_" + str(encoder.classes_[1]), 2 : "released_on_" + str(encoder.classes_[2]),3 : "released_on_" + str(encoder.classes_[3]),4 : "released_on_" + str(encoder.classes_[4]),5 : "released_on_" + str(encoder.classes_[5]),6 : "released_on_" + str(encoder.classes_[6])},inplace=True)

release_month_transformed = encoder.fit_transform(train["release_month"])
release_month = pd.DataFrame(release_month_transformed)
# Add to train dataframe
train = pd.concat([train,release_month], axis=1)

# Rename column name for easier use
train.rename(columns={0 : "released_on_" + "Jan", 1 : "released_on_" + "Feb", 2 : "released_on_" + "Mar",3 : "released_on_" + "Apr",4 : "released_on_" + "May",5 : "released_on_" + "Jun",6 : "released_on_" + "Jul",7 : "released_on_" + "Aug", 8 : "released_on_" + "Sep", 9 : "released_on_" + "Oct",10 : "released_on_" + "Nov",11 : "released_on_" + "Dec"},inplace=True)

#### runtime

In [ ]:
print(train["runtime"].isnull().values.sum(), "null values found before filling na" )
median = train["runtime"].median()
train["runtime"].fillna(median,inplace=True)
print(train["runtime"].isnull().values.sum(), "null values found after filling na" )
train.plot(x="runtime",y="revenue",kind="scatter")
plt.show()

#### Spoken language

In [ ]:
# Separate and extract country name from production_countries
train['spoken_languages_count'] = train['spoken_languages'].apply(lambda x: len(ast.literal_eval(x)) if x == x else 0)
train['all_spoken_languages'] = train['spoken_languages'].apply(lambda x: ",".join(sorted([i['name'] for i in ast.literal_eval(x)])) if x == x else '')

# Calculate mean revenue for these movies
revenue_means = {}
for value in train['all_spoken_languages'].unique():
    query = "all_spoken_languages=='" + value + "'"
    revenue_means[value] = round(train.query(query)['revenue'].mean(),2)

# Print the top 10 mean revenue
counter = Counter(revenue_means)
print ("TOP 10 revenue by spoken languages\n")
for v in counter.most_common(10):
    languages, mean_revenue = v
    print("Movie of spoken language " + languages + " has mean revenue " + str(mean_revenue))

Encode the month and release_dayofweek to become boolean value (e.g. release_on_Wed, release_on_June)

#### Status

In [ ]:
# Sum revenue of all released movie
print ("Released movie mean revenue = " + str(train.query("status=='Released'")['revenue'].mean()))
# Sum revenue of all rumoured movie
print ("Rumored movie mean revenue = " + str(train.query("status=='Rumored'")['revenue'].mean()))

Clearly, released movies have a much higher mean revenue than rumored movie, therefore, we will create a column isReleased to our train dataframe

In [ ]:
train["is_released"] = np.where(train["status"]=="Released",1,0)
train.drop(columns=["status"],inplace=True)

#### Keywords

In [ ]:
# Separate and extract country name from production_countries
train['keywords_count'] = train['Keywords'].apply(lambda x: len(ast.literal_eval(x)) if x == x else 0)
train['all_keywords'] = train['Keywords'].apply(lambda x: ",".join(sorted([i['name'] for i in ast.literal_eval(x)])) if x == x else '')

# Calculate mean revenue for these movies
revenue_means = {}
for value in train['all_keywords'].unique():
    query = 'all_keywords=="' + value + '"'
    revenue_means[value] = round(train.query(query)['revenue'].mean(),2)

# Print the top 10 mean revenue
counter = Counter(revenue_means)
print ("TOP 10 revenue by keywords\n")
for v in counter.most_common(10):
    keywords, mean_revenue = v
    print("Movie of Keywords " + keywords + " has mean revenue " + str(mean_revenue))

### Tim's part
Tim is working on data-preprocessing for column cast

In [ ]:
# Add your code here

### Shu's part
Shu is working on data-preprocessing for column crew

In [ ]:
# Add your code here

## Check correlations

In [ ]:
# Looking for correlations
corr_matrix = train.corr()
corr_matrix["revenue"].sort_values(ascending=False)


## Evaluation

In [ ]:
# X = 
y = train["revenue"]

# Normalize the data
min_max_scalar = preprocessing.MinMaxScaler()
X = min_max_scalar.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)

# Select model and fit
# model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

# Calculate RMSE
RMSE = math.sqrt(mean_squared_error(y_test,y_pred))
print("RMSE = " + str(round(RMSE,2)))